In [1]:
!pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.1.2%2Bcu121-cp310-cp310-win_amd64.whl (2473.9 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.2%2Bcu121-cp310-cp310-win_amd64.whl (5.6 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.1.2%2Bcu121-cp310-cp310-win_amd64.whl (4.0 MB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached fsspec-2023.12.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached numpy-1.26.3-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pillow-10.2.0-cp310-cp310-win_amd64.whl.metadata (9.9 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached pillow-10.2.0-cp310-cp310-win_amd64.whl (2.6 MB

In [2]:
import torch
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'Quadro M1200'

In [3]:
!pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.39.1-py3-none-win_amd64.whl

     ---------------------------------------- 0.0/137.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/137.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/137.9 MB ? eta -:--:--
     -------------------------------------- 0.0/137.9 MB 187.9 kB/s eta 0:12:15
     -------------------------------------- 0.1/137.9 MB 328.2 kB/s eta 0:07:01
     -------------------------------------- 0.2/137.9 MB 908.0 kB/s eta 0:02:32
     ---------------------------------------- 0.4/137.9 MB 1.6 MB/s eta 0:01:24
     ---------------------------------------- 0.8/137.9 MB 2.7 MB/s eta 0:00:51
     ---------------------------------------- 1.3/137.9 MB 3.7 MB/s eta 0:00:38
     ---------------------------------------- 1.6/137.9 MB 4.1 MB/s eta 0:00:34
      --------------------------------------- 2.0/137.9 MB 4.7 MB/s eta 0:00:30
      --------------------------------------- 2.5/137.9 MB 5.2 MB/s eta 0:00:26
      --------------------------------------- 3.0/137.9 MB 5.

In [5]:
!pip install trl transformers accelerate  einops peft -Uqqq
#!pip install bitsandbytes einops wandb -Uqqq

# Fine tuning TinyLlama for custom dataset in local environment

In [6]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset

C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Preparing data 

In [7]:
import pandas as pd

In [8]:
df=pd.read_csv("C:\\Users\\prompt\\Documents\\27Dec2023\\data_set_20dec2023.csv")

In [9]:
data_df=df.copy()

In [10]:
data_df["text"] = data_df[["question", "answer"]].apply(lambda x: "question: " + x["question"] + " answer: " + x["answer"], axis=1)

In [11]:
data_df.head()

,Unnamed: 0,question,answer,text
0,0,What is the notice period duration?,Notice period is 90 days,question: What is the notice period duration? ...
1,1,Is it possible to take leaves during Notice pe...,"Sick leaves can be taken, In case casual leave...",question: Is it possible to take leaves during...
2,2,Is it possible to lieu notice period?,Pay three month's basic salary in lieu of noti...,question: Is it possible to lieu notice period...
3,3,What is the process of Final settlement?,HR will complete the Full and Final Settlement...,question: What is the process of Final settlem...
4,4,What about Final settlement?,HR will complete the Full and Final Settlement...,question: What about Final settlement? answer:...


# configuring model parameters

In [12]:
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,           
    bnb_4bit_quant_type="nf4",    
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, 
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, 
    device_map="auto",  
    trust_remote_code=True, 
)

config.json: 100%|██████████| 607/607 [00:00<?, ?B/s] 
C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prompt\.cache\huggingface\hub\models--PY007--TinyLlama-1.1B-step-50K-105b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
model.safetensors: 100%|███████


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\bitsandbytes\libbitsandbytes_cuda122_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.2\bin\cudart64_12.dll
CUDA SETUP: Highest compute capability among GPUs detected: 5.0
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\bitsandbytes\libbitsandbytes_cuda122_nocublaslt.dll...


generation_config.json: 100%|██████████| 129/129 [00:00<?, ?B/s] 


# loading tokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) 
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 194kB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 8.74MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 3.68MB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<?, ?B/s] 


# Setting up GPU Device

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [15]:
device

'cuda'

# setting up PEFT parameters

In [16]:
# Setting arguments for low-rank adaptation 

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05 
lora_rank = 32 

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

# Setting up Training parameters

In [23]:
output_dir = "Jan_11_2024/tinyllama" # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2  
optim = "paged_adamw_32bit" 
save_strategy="steps" 
save_steps = 10 
logging_steps = 10  
learning_rate = 2e-3  
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 200     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    #push_to_hub=True,
    report_to='none'
)

In [24]:
data = Dataset.from_pandas(data_df)

# model finetuning or training

In [25]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

Map: 100%|██████████| 51/51 [00:00<00:00, 4108.27 examples/s]


In [26]:
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.269900
20,1.227400
30,0.732900
40,0.479500
50,0.343500
60,0.277500
70,0.227500
80,0.217200
90,0.202300
100,0.184100


C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\prompt\anaconda3\envs\Jan_11_2024\lib\s

TrainOutput(global_step=200, training_loss=0.3838002634048462, metrics={'train_runtime': 1258.1605, 'train_samples_per_second': 0.954, 'train_steps_per_second': 0.159, 'total_flos': 282704325230592.0, 'train_loss': 0.3838002634048462, 'epoch': 23.53})

In [28]:
def get_response(query):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return text_output

# inference or model validation

In [30]:
get_response("question: what is notice period? answer:")

"question: what is notice period? answer: Notice period is 90 days from the date of exiting the service contract, or you can leave it in default.\n3. What about termination process? answer: company reserves the right to terminate employee's services by either giving notice or paying basic salary in lieu of notice period, or providing notice at the last day of work (non-working hours) for a paid standard salary, and may also choose to end your employment at the end of the current year / month, in accordance with the provisions contained therein, to make way for new employees to be created during that same calendar year / month. You agree not to file any complaints against the Company within three months from the date on which you were employed by the Company. In case you are terminated due to your conduct to become unreasonable, then he/she will charge you the normal rate of salary as well as other applicable taxes, fees and restitution costs to the extent they have been charged to his/

In [31]:
get_response("question: What is the notice period duration? answer:")

"question: What is the notice period duration? answer: Notice period is 90 days.\n3. The company reserves the right to change the projected timeline, or both, of the projected timelines may be changed at any time without notification in writing.\n4. All employees are required to follow the Company's policies and procedures when working on projects.\n5. In case a team member is terminated for reasons beyond his control, the employee will be entitled to receive benefits as well as other forms of compensation to the extent that such form of compensation was awarded to him/her by the employer in last full payroll year.\n6. Employees who have completed five years from the date of hire shall not be eligible to claim the pension under the provisions of the Workers' Compensation Act, 1972 until he/she receives payment in lieu of tenure.\n7. An employee would be eligible to retirement if he/ she has attained age 55 with no workouts to his/ her credit history during the preceding twelve months.\

In [32]:
get_response("question: is it possible to lieu notice period? answer:")

"question: is it possible to lieu notice period? answer: Pay three month's basic salary in lieu of notice period. Answer: While working with the company for the reporting year, the employee may be paid any amount to the employees full and effective communication with the employee on the performance review and other documents required by the company.\nThe Company reserves the right to change or add to the terms and conditions contained herein, including but not limited to the number of days worked per work week, bonus payable, holiday allowances and other benefits received from the Company during the term, as well as the total compensation paid to a worker who has been employed by the Company 30 continuous weeks (or longer) and pays off his normal earnings in full, will be as follows: - Rs. 15000 / month + bonuses/hrs earned; 2 x pts. per day ; 4 x pts. per night Will this increase in rate of pay would be taken into account while calculating earning ? Will you enable access to all the p

In [1]:
!pip freeze

accelerate==0.26.0
aiohttp==3.9.1
aiosignal==1.3.1
anyio @ file:///C:/ci/anyio_1644481856696/work/dist
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci/argon2-cffi-bindings_1644569876605/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru @ file:///C:/b/abs_e0hjkvwwb5/croot/async-lru_1699554572212/work
async-timeout==4.0.3
attrs @ file:///C:/b/abs_35n0jusce8/croot/attrs_1695717880170/work
Babel @ file:///C:/b/abs_a2shv_3tqi/croot/babel_1671782804377/work
beautifulsoup4 @ file:///C:/b/abs_0agyz1wsr4/croot/beautifulsoup4-split_1681493048687/work
bitsandbytes @ https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.39.1-py3-none-win_amd64.whl#sha256=7377d5f70aeb80d1b64dd3835c7079b9a18891613f02513e1dadf651ddb626dd
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
Brotli @ file:///C:/Windows/Temp/abs_63l7912z0e/croots/recipe/brotli-split_1659616056